In [25]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from scipy.stats import mode

In [2]:
test = pd.read_csv('data//formatted_data//test.csv')
train = pd.read_csv('data//formatted_data//train.csv')

In [26]:
'''
Steps
    1. Hypothesis
    2. Data Exploration     
    3. Data Cleaning
    4. Featuring Engineering
'''

# Data Exploration
#   1. Merging Test and Train
train['source'] = 'train'
test['source'] = 'test'
data = pd.concat([train,test], ignore_index=True, sort=False)
print(train.shape, test.shape, data.shape)

#   2. Exploring null values
data.apply(lambda x: sum(x.isnull()))

#   3. Exploring overall data stats
data.describe()

#   4. Exploring unique data in each column (especially useful for categorical columns)
data.apply(lambda x: len(x.unique()))
#     data.dtypes can help show which columns are categorical

#   5. Filter categorical variables (excluding ID cols and source)
cat_cols = [x for x in data.dtypes.index if data.dtypes[x]=='object']
cat_cols = [x for x in cat_cols if x not in ['Item_Identifier','Outlet_Identifier','source']]

#   6. Print frequency of categories
for col in cat_cols:
    print('\nFrequency of Categories for varible %s'%col)
    print(data[col].value_counts())

#   7. Exploration Findings/Notes
'''
General Description: there are 1559 items sold at 10 different stores
1. Null Value Columns: [Item_Weight:2439, Outlet_Size: 4016, Item_Outlet_Sales:5681]
2. Stats Description: 
     a. May want to standardize year column in different format other than 1985 to 2009.
     b. Item_Visibility has a min value of zero. This makes no practical sense because when a product is being sold in a store, the visibility cannot be 0.
3. Category Issues
     a. Item_Fat_Content: ([LF,low fat, Low Fat], [Regular, reg]) *should only be two distinct categories*
     b. Outlet_Type: *May need to combine supermarket types 1,2,3*
     c. Item_Type: Not all categories have substantial numbers. It looks like combining them can give better results.
     
'''

#Data Cleaning

#   8. Null Value Replacement for Item Weight
#     a. Determine the average weight per item
item_avg_weight = data.pivot_table(values='Item_Weight', index='Item_Identifier')
#     b. Get a boolean variable specifying missing Item_Weight values
miss_bool = data['Item_Weight'].isnull()
#     c. Impute data and check #missing values before and after imputation to confirm
print('Original #missing: %d'%sum(miss_bool))
data.loc[miss_bool,'Item_Weight'] = data.loc[miss_bool, 'Item_Identifier'].apply(lambda x: item_avg_weight.loc[item_avg_weight.index==x,'Item_Weight'][0])
print('Final #missing: %d'% sum(data['Item_Weight'].isnull()))

#   9. Null Value Replacement for Outlet size
#     a. Determine the mode outlet size per outlet type
outlet_size_mode = data[pd.notna(data['Outlet_Size'])==True].pivot_table(values='Outlet_Size', index='Outlet_Type',aggfunc=(lambda x:mode(x).mode[0]))
print('Mode for each Outlet_Type:')
print(outlet_size_mode)
#     b. Get a boolean variable specifying missingOutlet_Size values
miss_bool = data['Outlet_Size'].isnull()
#     c. Impute data and check #missing values before and after imputation to confirm
print('\nOriginal #missing: %d'%sum(miss_bool))
data.loc[miss_bool,'Outlet_Size'] = data.loc[miss_bool, 'Outlet_Type'].apply(lambda x: outlet_size_mode.loc[outlet_size_mode.index==x,'Outlet_Size'][0])
print('Final #missing: %d'% sum(data['Item_Weight'].isnull()))


#Feature Engineering
#   10. Consider Combining Outlet_Type
#     a. Exploration (keeping outlet type as is because significant difference between different types)
data.pivot_table(values='Item_Outlet_Sales', index='Outlet_Type')

#   11. Modify Item Visibility
#     a. Determine average visibility of a product
visibility_avg = data.pivot_table(values='Item_Visibility', index='Item_Identifier')
#     b. Impute 0 values with mean visibility of that product:
miss_bool = (data['Item_Visibility']==0)
print('Number of 0 values initially: %d'%sum(miss_bool))
data.loc[miss_bool,'Item_Visibility'] = data.loc[miss_bool,'Item_Identifier'].apply(lambda x: visibility_avg.loc[visibility_avg.index==x,'Item_Visibility'][0])
print('Number of 0 values after modification: %d'%sum(data['Item_Visibility']==0))
#     c. Creating modifided Item Visibility value that is compared to the mean visibility across all stores
data['Item_Visibility_MeanRatio'] = data.apply(lambda x: x['Item_Visibility']/visibility_avg.loc[visibility_avg.index==x['Item_Identifier'],'Item_Visibility'][0], axis=1)
print(data['Item_Visibility_MeanRatio'].describe())

#   12. Create New Column for Item Type (!!!! Should Modify to group better !!!!)
#     a. Get the first two charcters of ID and rename to more intuitive categories
data['Item_Type_Combined'] = data['Item_Identifier'].apply(lambda x: x[0:2])
data['Item_Type_Combined'] = data['Item_Type_Combined'].map({'FD':'Food',
                                                            'NC':'Non-Consumable',
                                                            'DR':'Drinks'})
data['Item_Type_Combined'].value_counts()

#   13. Creating a new column depicting the years of operation
data['Outlet_Years'] = 2013 - data['Outlet_Establishment_Year']
data['Outlet_Years'].describe()

#   14. Modifying Item_Fat Content Categories
print('Original Categories:')
print(data['Item_Fat_Content'].value_counts())
print('\nModified Categories:')
data['Item_Fat_Content'] = data['Item_Fat_Content'].replace({'LF':'Low Fat',
                                                            'reg':'Regular',
                                                            'low fat': 'Low Fat'})
#Makes sure that non-consumable items don't have fat content listed
data.loc[data['Item_Type_Combined']=='Non-Consumable','Item_Fat_Content'] = 'Non-Edible'
print(data['Item_Fat_Content'].value_counts())

#   15. One-Hot Encoding Categorical Variables
lb = preprocessing.LabelBinarizer()
le = preprocessing.LabelEncoder()
#     a. New vairable for outlet
data['Outlet'] = le.fit_transform(data['Outlet_Identifier'].values)

#     b. One Hot Encoding Remaining Categorical Variables
var_mod = ['Item_Fat_Content', 'Outlet_Location_Type','Outlet_Size','Item_Type_Combined','Outlet_Type','Outlet']
for i in var_mod:
    data[i] = le.fit_transform(data[i])
    
data[]
print(data.dtypes)
#     c. 

(8523, 13) (5681, 12) (14204, 13)

Frequency of Categories for varible Item_Fat_Content
Low Fat    8485
Regular    4824
LF          522
reg         195
low fat     178
Name: Item_Fat_Content, dtype: int64

Frequency of Categories for varible Item_Type
Fruits and Vegetables    2013
Snack Foods              1989
Household                1548
Frozen Foods             1426
Dairy                    1136
Baking Goods             1086
Canned                   1084
Health and Hygiene        858
Meat                      736
Soft Drinks               726
Breads                    416
Hard Drinks               362
Others                    280
Starchy Foods             269
Breakfast                 186
Seafood                    89
Name: Item_Type, dtype: int64

Frequency of Categories for varible Outlet_Size
Medium    4655
Small     3980
High      1553
Name: Outlet_Size, dtype: int64

Frequency of Categories for varible Outlet_Location_Type
Tier 3    5583
Tier 2    4641
Tier 1    3980
Name: Out

C:\Anaconda3\lib\site-packages\scipy\stats\stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


Final #missing: 0
Number of 0 values initially: 879
Number of 0 values after modification: 0
count    14204.000000
mean         1.061884
std          0.235907
min          0.844563
25%          0.925131
50%          0.999070
75%          1.042007
max          3.010094
Name: Item_Visibility_MeanRatio, dtype: float64
Original Categories:
Low Fat    8485
Regular    4824
LF          522
reg         195
low fat     178
Name: Item_Fat_Content, dtype: int64

Modified Categories:
Low Fat       6499
Regular       5019
Non-Edible    2686
Name: Item_Fat_Content, dtype: int64
Item_Identifier               object
Item_Weight                  float64
Item_Fat_Content               int64
Item_Visibility              float64
Item_Type                     object
Item_MRP                     float64
Outlet_Identifier             object
Outlet_Establishment_Year      int64
Outlet_Size                    int64
Outlet_Location_Type           int64
Outlet_Type                    int64
Item_Outlet_Sales     

In [35]:
x = lb.fit_transform(data.Outlet_Type)

In [36]:
x

array([[0, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 1, 0, 0],
       ...,
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0]])

In [17]:
lb = preprocessing.LabelBinarizer()
lb.fit_transform(['yes', 'no', 'no', 'yes','turkey','turkey'])

array([[0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 0]])

In [31]:
data.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,source,Item_Visibility_MeanRatio,Item_Type_Combined,Outlet_Years,Outlet
0,FDA15,9.30,0,0.016047,Dairy,249.8092,OUT049,1999,1,0,1,3735.1380,train,0.931078,1,14,9
1,DRC01,5.92,2,0.019278,Soft Drinks,48.2692,OUT018,2009,1,2,2,443.4228,train,0.933420,0,4,3
2,FDN15,17.50,0,0.016760,Meat,141.6180,OUT049,1999,1,0,1,2097.2700,train,0.960069,1,14,9
3,FDX07,19.20,2,0.017834,Fruits and Vegetables,182.0950,OUT010,1998,2,2,0,732.3800,train,1.000000,1,15,0
4,NCD19,8.93,1,0.009780,Household,53.8614,OUT013,1987,0,2,1,994.7052,train,1.000000,2,26,1


In [30]:
data['Outlet_Type'].unique()

array([1, 2, 0, 3], dtype=int64)

In [ ]:
#     a. New vairable for outlet
data['Outlet'] = le.fit_transform(data['Outlet_Identifier'].values)

#     b. One Hot Encoding Remaining Categorical Variables
var_mod = ['Item_Fat_Content', 'Outlet_Location_Type','Outlet_Size','Item_Type_Combined','Outlet_Type','Outlet']
for i in var_mod:
    data[i] = le.fit_transform(data[i])
print(data.dtypes)
#     c. 